In [1]:
import asyncio
import websockets
import requests
import json
import nest_asyncio
from collections import defaultdict

In [53]:
nest_asyncio.apply()

BINANCE_WS_URL = "wss://fstream.binance.com/stream?streams=ethusdtd@depth,adausdt@depth"
BINANCE_REST_URL = "https://fapi.binance.com/fapi/v1/depth?symbol=BTCUSDT&limit=1000"

In [5]:
async def process_depth_updates():
    async with websockets.connect(BINANCE_WS_URL) as websocket:
        order_book = await get_snapshot()
        last_update_id = order_book["lastUpdateId"]
        print(f"Initial snapshot obtained with lastUpdateId: {last_update_id}")
        
        async for message in websocket:
            data = json.loads(message)["data"]
            if data["u"] < last_update_id:
                continue  # Ignore events before the snapshot
            
            if not (data["U"] <= last_update_id + 1 <= data["u"]):
                print("Inconsistent update sequence, reloading snapshot...")
                order_book = await get_snapshot()
                last_update_id = order_book["lastUpdateId"]
                continue
            
            for price, qty in data["b"]:
                if float(qty) == 0:
                    order_book["bids"].pop(price, None)  # Remove price level
                else:
                    order_book["bids"][price] = float(qty)  # Update bid
            
            for price, qty in data["a"]:
                if float(qty) == 0:
                    order_book["asks"].pop(price, None)  # Remove price level
                else:
                    order_book["asks"][price] = float(qty)  # Update ask
            
            last_update_id = data["u"]  # Update last processed ID
            print(f"Updated order book at event {data['E']}")

In [7]:
async def get_snapshot():
    response = requests.get(BINANCE_REST_URL)
    snapshot = response.json()
    print(snapshot.keys())
    order_book = {
        "bids": {price: float(qty) for price, qty in snapshot["bids"]},
        # "bids": snapshot['bids'],
        # "asks": snapshot['asks'],
        "asks": {price: float(qty) for price, qty in snapshot["asks"]},
        "lastUpdateId": snapshot["lastUpdateId"]
    }
    return order_book

In [9]:
# Run the function
# task = asyncio.create_task(process_depth_updates())

In [11]:
task.cancel()

NameError: name 'task' is not defined

In [55]:
async def process_websocket():
    async with websockets.connect(BINANCE_WS_URL) as websocket:
        order_book = await get_snapshot()
        last_update_id = order_book["lastUpdateId"]
        print(f"Initial snapshot obtained with lastUpdateId: {last_update_id}")
        
        async for message in websocket:
            data = json.loads(message)["data"]
            if data["u"] < last_update_id:
                continue  # Ignore events before the snapshot
            
            if not (data["U"] <= last_update_id + 1 <= data["u"]):
                print("Inconsistent update sequence, reloading snapshot...")
                order_book = await get_snapshot()
                last_update_id = order_book["lastUpdateId"]
                continue
            
            for price, qty in data["b"]:
                if float(qty) == 0:
                    order_book["bids"].pop(price, None)  # Remove price level
                else:
                    order_book["bids"][price] = float(qty)  # Update bid
            
            for price, qty in data["a"]:
                if float(qty) == 0:
                    order_book["asks"].pop(price, None)  # Remove price level
                else:
                    order_book["asks"][price] = float(qty)  # Update ask
            
            last_update_id = data["u"]  # Update last processed ID
            print(f"Updated order book at event {data['E']}")

In [69]:
import websocket
import json

In [79]:
BINANCE_WS_URL = "wss://fstream.binance.com/stream"

def on_message(ws, message):
    data = json.loads(message)
    stream = data["stream"]  # Identifies the stream (e.g., "btcusdt@depth")
    order_book = data["data"]  # Order book data
    
    print(f"Stream: {stream}")
    print(order_book)

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    print("WebSocket connection opened")
    
    # Subscribe to multiple streams after connecting
    payload = {
        "method": "SUBSCRIBE",
        "params": ["adausdt@depth", "ethusdt@depth"],  # List of streams
        "id": 1
    }
    ws.send(json.dumps(payload))  # Send the request to Binance

# Initialize WebSocket connection
ws = websocket.WebSocketApp(
    BINANCE_WS_URL,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close,
    on_open=on_open
)


In [81]:
try:
    ws.run_forever()
except KeyboardInterrupt:
    print("Stopping WebSocket...")
    ws.close()

WebSocket connection opened
Error: 'stream'
Stream: ethusdt@depth
{'e': 'depthUpdate', 'E': 1738500379852, 'T': 1738500379851, 's': 'ETHUSDT', 'U': 6621956868673, 'u': 6621956883480, 'pu': 6621956868444, 'b': [['200.00', '30.942'], ['2934.72', '0.063'], ['3008.09', '3.341'], ['3026.43', '0.000'], ['3045.81', '0.693'], ['3045.82', '0.087'], ['3049.93', '13.683'], ['3051.22', '4.682'], ['3052.25', '0.023'], ['3053.30', '0.206'], ['3053.41', '21.836'], ['3055.05', '1.902'], ['3055.12', '2.315'], ['3055.17', '12.617'], ['3055.21', '7.382'], ['3055.34', '44.870'], ['3055.36', '20.221'], ['3055.38', '0.000'], ['3055.81', '0.014'], ['3056.01', '169.507'], ['3056.10', '3.997'], ['3056.14', '16.720'], ['3056.18', '16.211'], ['3056.28', '0.465'], ['3056.30', '2.341'], ['3056.47', '0.171'], ['3056.48', '0.009'], ['3056.50', '5.626'], ['3056.51', '4.369'], ['3056.52', '0.000'], ['3056.54', '0.177'], ['3056.56', '19.625'], ['3056.57', '5.026'], ['3056.58', '0.350'], ['3056.60', '14.858'], ['3056.61